In [1]:
cd /home

/home


In [2]:
import matplotlib.pyplot as plt 

import pandas as pd
import numpy as np
import toml
import wandb
import toml
import glob
import os

from src.utils import get_metrics

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.3) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-09-06 15:18:31.034997: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [25]:
folder = './presentation/experiments/astromer_1/results/'
exp_folders = glob.glob(os.path.join(folder, '*', '*', 'pretraining'))
exp_folders

['./presentation/experiments/astromer_1/results/paper/2023-09-03_04-30-55/pretraining',
 './presentation/experiments/astromer_1/results/paper/2023-09-05_15-39-27/pretraining']

In [26]:
metric_names= ['loss', 'r_square', 'rmse', 'bce', 'accuracy']

rows = []
for expdir in exp_folders:
    config_path = os.path.join(expdir, 'config.toml')
    log_dir = os.path.join(expdir, 'logs')
    
    with open(config_path, 'r') as f:
        config = toml.load(f)

        val_metrics = get_metrics(os.path.join(log_dir, 'validation'), 
                                  metric_name='loss')
        best_epoch = val_metrics['value'].astype(float).argmin() 


        for index, metric in enumerate(metric_names):
            try:
                train_metrics = get_metrics(os.path.join(log_dir, 'train'), metric_name=metric)
                valid_metrics = get_metrics(os.path.join(log_dir, 'validation'), metric_name=metric)

                config[metric] = np.round(train_metrics.iloc[best_epoch]['value'], 3)
                config[metric+'_val'] = np.round(valid_metrics.iloc[best_epoch]['value'], 3)        
            except:
                continue
                
        rows.append(pd.Series(config))

results = pd.concat(rows, axis=1).T

In [32]:
results['checkpoint'].iloc[1]

'./presentation/experiments/astromer_1/results/paper/2023-09-03_04-30-55/pretraining/'